In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf 

from baseline import Baseline
from system_t import System_T

import warnings
warnings.filterwarnings('ignore')

# Load Fashion-MNIST dataset from keras

In [2]:
fashion = tf.keras.datasets.fashion_mnist
(fashion_train_images,fashion_train_labels), (fashion_test_images, fashion_test_labels) = fashion.load_data()

fashion_train_labels = fashion_train_labels
fashion_test_labels = fashion_test_labels
fashion_train_images = fashion_train_images.reshape((60000, 28, 28, 1)) / 255.0
fashion_test_images = fashion_test_images.reshape((10000, 28, 28, 1)) / 255.0

train_images = fashion_train_images
train_labels = fashion_train_labels

In [3]:
def shuffle(data, label):
    shuffle = np.arange(len(data))
    np.random.shuffle(shuffle)
    data = data[shuffle]
    label = label[shuffle]
    return data, label

num_class = len(np.unique(train_labels))
print("Number of slices : %d" % num_class)

y_train_one_hot = tf.keras.utils.to_categorical(train_labels)
mixed_data = (train_images, y_train_one_hot)

initial_data_array = []
val_data_dict = []
add_data_dict = []
    
val_data_num = 500

for i in range(num_class):
    data_num = int(500 / (i + 1)**(1/2))
    initial_data_array.append(data_num)
    idx = np.argmax(mixed_data[1], axis=1) == i

    val_data_dict.append((mixed_data[0][idx][data_num:data_num+val_data_num], mixed_data[1][idx][data_num:data_num+val_data_num]))
    add_data_dict.append((mixed_data[0][idx][data_num+val_data_num:], mixed_data[1][idx][data_num+val_data_num:]))

    if i == 0:
        train_data = mixed_data[0][idx][:data_num]
        train_label = mixed_data[1][idx][:data_num]
        val_data = mixed_data[0][idx][data_num:data_num+val_data_num]
        val_label = mixed_data[1][idx][data_num:data_num+val_data_num]
    else:
        train_data = np.concatenate((train_data, mixed_data[0][idx][:data_num]), axis=0)
        train_label = np.concatenate((train_label, mixed_data[1][idx][:data_num]), axis=0) 
        val_data = np.concatenate((val_data, mixed_data[0][idx][data_num:data_num+val_data_num]), axis=0)
        val_label = np.concatenate((val_label, mixed_data[1][idx][data_num:data_num+val_data_num]), axis=0)   
    
train_data, train_label = shuffle(train_data, train_label)
print(train_data.shape, train_label.shape, num_class)

Number of slices : 10
(2506, 28, 28, 1) (2506, 10) 10


# Define slices

In [4]:
slice_desc = []
a = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

# Mixed mnist
#a = ["Digit 0", "Digit 1","Digit 2","Digit 3","Digit 4","Digit 5","Digit 6","Digit 7","Digit 8","Digit 9",
#    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

for i in range(num_class):
    slice_desc.append('Slice: %s' % (a[i]))

# System T Demo on Fashion-MNIST

## Use 3000 budget, lambda=0.1, "Moderate" strategy

In [5]:
cost_func = [1] * num_class

budget = 3000
lr = 0.0005
method = 'Moderate'

st = System_T((train_data, train_label), (val_data, val_label), val_data_dict, initial_data_array, num_class, add_data_dict)
st.selective_collect(budget=budget, k=10, batch_size=32, lr = lr, epochs=2000, cost_func=cost_func, 
                 Lambda=0.1, num_iter=5, slice_desc=slice_desc, strategy=method, show_figure=False)

======= Collect Data =======
[  0   0 250  33 234  16 470 103   0   0]
Total Cost: 1106, Remaining Budget: 1894
======= Collect Data =======
[131   0 261 259   0  41 316 109 116   0]
Total Cost: 1233, Remaining Budget: 661
======= Collect Data =======
[  0   0   0   0 498  10  97   0   0  56]
Total Cost: 661, Remaining Budget: 0

======= Performance =======
[131.   0. 511. 292. 732.  67. 883. 212. 116.  56.]
Number of iteration: 3
Strategy: Moderate, C: 0.1, Budget: 3000
Loss: 0.30776 (0.00574), Average EER: 0.15761, Max EER: 0.38573



# Baseline: Uniform

In [6]:
uni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
uni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 3000
======= Collect Data =======
[300 300 300 300 300 300 300 300 300 300]
======= Performance =======
Loss: 0.31844 (0.00512), Average EER: 0.20004, Max EER: 0.54026



# Baseline: Waterfilling

In [7]:
wf = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Waterfilling')
wf.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Waterfilling, Budget: 3000
======= Collect Data =======
[ 51 198 263 301 328 347 362 374 384 392]
======= Performance =======
Loss: 0.31283 (0.00387), Average EER: 0.20770, Max EER: 0.58765

